In [1]:
import requests, bs4
import pandas as pd
import numpy as np
import asyncio
from lxml import html
import time
import aiohttp
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

# url 입력
#servicekey 뒤에 자기 개인키 입력
url = 'http://apis.data.go.kr/6260000/BusanBIMS/busStopList?serviceKey=hI8BcsbxIOje%2FVupnOrk2Lwzag7%2F2bKTR9zxN3wwTFIuWoCmnVfhtuABpDzxHKLweq6HykobkTpmsrJRt8e4oA%3D%3D&pageNo=1&numOfRows=8600'

# url 불러오기
response = requests.get(url)

#데이터 값 출력해보기
contents = response.text

In [2]:
xmlobj = bs4.BeautifulSoup(contents, 'lxml-xml')
rows = xmlobj.findAll('item')

# 모든 행과 열의 값을 모아 매트릭스로 만들어보자.
rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없다.)
        if i == 0:
            nameList.append(columns[j].name)
        # 컬럼값은 모든 행의 값을 저장해야한다.    
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)

    
result = pd.DataFrame(rowList,columns=nameList)

result['arsno2'] = result['arsno']
result['gpsx2'] = result['gpsx']
result['gpsy2'] = result['gpsy']
result['stoptype2'] = result['stoptype']
result.loc[result.stoptype.isnull() == True,'stoptype2'] = result.loc[result.stoptype.isnull() == True,'gpsy2']
result.loc[result.stoptype.isnull() == True,'gpsy2'] = result.loc[result.stoptype.isnull() == True,'gpsx2']
result.loc[result.stoptype.isnull() == True,'gpsx2'] = result.loc[result.stoptype.isnull() == True,'arsno2']
result.loc[result.stoptype.isnull() == True,'arsno2'] = np.nan
result1 = result.drop(['arsno', 'gpsx', 'gpsy', 'stoptype'],axis = 1)
busid = list(result1['bstopid'])

In [124]:
async def bus(session,number):
    count = 0
    try:
        async with session.get('http://apis.data.go.kr/6260000/BusanBIMS/stopArrByBstopid?serviceKey=hI8BcsbxIOje%2FVupnOrk2Lwzag7%2F2bKTR9zxN3wwTFIuWoCmnVfhtuABpDzxHKLweq6HykobkTpmsrJRt8e4oA%3D%3D&bstopid={0}'.format(number)) as resp:
            contents = await resp.text()
            xmlobj = bs4.BeautifulSoup(contents, 'lxml-xml')
            rows = xmlobj.findAll('item')
             # 모든 행과 열의 값을 모아 매트릭스로 만들어보자.
            rowList = []
            nameList = []
            columnList = []

            rowsLen = len(rows)
            for i in range(0, rowsLen):
                columns = rows[i].find_all(['arsno','bstopid','lineno','nodenm','nodenm','gpsx','gpsy','bustype','lineid'])
                columnsLen = len(columns)
                for j in range(0, columnsLen):
                    # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없다.)
                    if i == 0:
                        nameList.append(columns[j].name)
                    # 컬럼값은 모든 행의 값을 저장해야한다.    
                    eachColumn = columns[j].text
                    columnList.append(eachColumn)
                rowList.append(columnList)
                columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)
            result = pd.DataFrame(rowList,columns=nameList)
            return result
    except:
        count = count +  1
        if count % 100 == 0:
            print(count)

async def busapi(numbers):
    results = pd.DataFrame()
    async with aiohttp.ClientSession() as session:
        results = results.append(await asyncio.gather(*[bus(session, number) for number in numbers]))
        return results
    
s = time.time()
b = await busapi(busid2)
print(time.time()-s)    
len(b.groupby('bstopid').sum())

9.350449085235596


KeyError: 'bstopid'

In [103]:
8517 - 8261

256

In [ ]:
result1[~(result1['bstopid'].isin(total1['bstopid']))]

,bstopid,bstopnm,arsno2,gpsx2,gpsy2,stoptype2
3639,215060202,부산역,70001,129.040292,35.11454554,일반
3640,180340101,부산박물관,70002,129.0934465,35.13005122,일반
3641,199330101,광안리해수욕장,70003,129.118294,35.15320698,일반
3642,186280101,동백섬,70004,129.1514109,35.15768313,일반
3643,185760101,해운대해수욕장,70005,129.161013333333,35.159595,일반
...,...,...,...,...,...,...
8416,509510000,아랫매리,NaN,128.966877507954,35.318899340784,일반
8417,509520000,아랫매리,NaN,128.96701936482,35.319081769787,일반
8418,509530000,월촌2교,NaN,128.992177259637,35.284168975509,일반
8419,509540000,월촌2교,NaN,128.991974353454,35.284185835611,일반


In [102]:
total1 = pd.concat([try1,try2,try3],axis = 0)
len(total1.groupby('bstopid').sum())

8261

In [105]:
busid2 = result1[~(result1['bstopid'].isin(total1['bstopid']))]

In [106]:
busid2 = busid2.reset_index(drop = True)
busid2 = busid2['bstopid']
busid2 = list(busid2)
len(busid2)

256

In [122]:
none = result1[~(result1['bstopid'].isin(total1['bstopid']))]
none

,bstopid,bstopnm,arsno2,gpsx2,gpsy2,stoptype2
3639,215060202,부산역,70001,129.040292,35.11454554,일반
3640,180340101,부산박물관,70002,129.0934465,35.13005122,일반
3641,199330101,광안리해수욕장,70003,129.118294,35.15320698,일반
3642,186280101,동백섬,70004,129.1514109,35.15768313,일반
3643,185760101,해운대해수욕장,70005,129.161013333333,35.159595,일반
...,...,...,...,...,...,...
8416,509510000,아랫매리,NaN,128.966877507954,35.318899340784,일반
8417,509520000,아랫매리,NaN,128.96701936482,35.319081769787,일반
8418,509530000,월촌2교,NaN,128.992177259637,35.284168975509,일반
8419,509540000,월촌2교,NaN,128.991974353454,35.284185835611,일반


In [129]:
total1.loc[total1.nodenm == '하리']

,arsno,bstopid,lineno,nodenm,gpsx,gpsy,bustype,lineid
0,04095,171050102,1006,하리,129.0804283333,35.0687466667,급행버스,5201006000
1,04095,171050102,101,하리,129.0804283333,35.0687466667,일반버스,5200101000
2,04095,171050102,186,하리,129.0804283333,35.0687466667,일반버스,5200186000
3,04095,171050102,30,하리,129.0804283333,35.0687466667,일반버스,5200030000
4,04095,171050102,66,하리,129.0804283333,35.0687466667,일반버스,5200066000
5,04095,171050102,8,하리,129.0804283333,35.0687466667,일반버스,5200008000
6,04095,171050102,88(A),하리,129.0804283333,35.0687466667,일반버스,5200088000
0,04096,171060201,1006,하리,129.080833314296,35.067928748488,급행버스,5201006000
1,04096,171060201,101,하리,129.080833314296,35.067928748488,일반버스,5200101000
2,04096,171060201,186,하리,129.080833314296,35.067928748488,일반버스,5200186000


In [128]:
n = pd.read_csv('C:\\Users\\user\\Desktop\\none_bus_busan.csv',encoding = 'euc-kr')
n

,bstopid,bstopnm,arsno2,gpsx2,gpsy2,stoptype2
0,215060202,부산역,70001.0,129.040292,35.114546,일반
1,180340101,부산박물관,70002.0,129.093446,35.130051,일반
2,199330101,광안리해수욕장,70003.0,129.118294,35.153207,일반
3,186280101,동백섬,70004.0,129.151411,35.157683,일반
4,185760101,해운대해수욕장,70005.0,129.161013,35.159595,일반
...,...,...,...,...,...,...
67,500790000,하리,NaN,129.235665,35.312416,마을
68,502400000,기장시장,NaN,129.216299,35.242958,일반
69,502410000,대변항.멸치테마광장,NaN,129.228172,35.224749,일반
70,502420000,아난티코브(힐튼부산),NaN,129.228822,35.197429,일반
